In [0]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import random
#torch.set_default_tensor_type('torch.cuda.FloatTensor')


In [0]:
def creatData(length):
  que = []
  ans = []
  for i in range(length):
    x = random.randint(1,1000)
    y = random.randint(1,999)
    a = x+y
    str1 = str(x)+'+'+str(y)
    str2 = str(a)
    while len(str1)<8: 
      str1 = str1+' '
    while len(str2)<4: 
      str2 = str2+' '
    que.append(str1)
    ans.append(str2)
  return que,ans

In [4]:
print(que)
print(ans)

['114+815 ', '99+442  ', '42+215  ', '501+882 ', '126+121 ', '873+570 ', '670+18  ', '119+889 ', '870+213 ', '289+688 ', '460+416 ', '400+391 ', '94+472  ', '894+668 ', '719+707 ', '902+127 ', '967+863 ', '355+250 ', '990+213 ', '605+182 ', '877+253 ', '829+333 ', '76+765  ', '47+573  ', '658+457 ', '779+766 ', '443+812 ', '853+312 ', '606+152 ', '344+663 ', '578+56  ', '327+122 ', '503+182 ', '110+897 ', '197+580 ', '750+997 ', '96+888  ', '654+926 ', '277+918 ', '964+69  ', '853+470 ', '668+293 ', '949+724 ', '526+564 ', '250+972 ', '400+81  ', '889+692 ', '296+348 ', '115+169 ', '835+461 ', '145+286 ', '38+976  ', '845+126 ', '717+991 ', '301+413 ', '840+800 ', '592+849 ', '300+314 ', '621+111 ', '841+707 ', '310+810 ', '172+867 ', '96+38   ', '212+324 ', '888+210 ', '376+555 ', '859+884 ', '726+648 ', '864+161 ', '639+466 ', '361+344 ', '849+420 ', '185+393 ', '519+492 ', '674+845 ', '925+506 ', '573+925 ', '933+480 ', '474+525 ', '548+483 ', '319+941 ', '851+667 ', '39+783  ', '91

In [0]:
dict1 = {' ': [1,0,0,0,0,0,0,0,0,0,0,0],
        '+': [0,1,0,0,0,0,0,0,0,0,0,0],
        '0': [0,0,1,0,0,0,0,0,0,0,0,0],
        '1': [0,0,0,1,0,0,0,0,0,0,0,0],
        '2': [0,0,0,0,1,0,0,0,0,0,0,0],
        '3': [0,0,0,0,0,1,0,0,0,0,0,0],
        '4': [0,0,0,0,0,0,1,0,0,0,0,0],
        '5': [0,0,0,0,0,0,0,1,0,0,0,0],
        '6': [0,0,0,0,0,0,0,0,1,0,0,0],
        '7': [0,0,0,0,0,0,0,0,0,1,0,0],
        '8': [0,0,0,0,0,0,0,0,0,0,1,0],
        '9': [0,0,0,0,0,0,0,0,0,0,0,1]}

# dict2 = {[1,0,0,0,0,0,0,0,0,0,0,0]:' ',
#          [0,1,0,0,0,0,0,0,0,0,0,0]:'+',
#          [0,0,1,0,0,0,0,0,0,0,0,0]:'0',
#          [0,0,0,1,0,0,0,0,0,0,0,0]:'1',
#          [0,0,0,0,1,0,0,0,0,0,0,0]:'2',
#          [0,0,0,0,0,1,0,0,0,0,0,0]:'3',
#          [0,0,0,0,0,0,1,0,0,0,0,0]:'4',
#          [0,0,0,0,0,0,0,1,0,0,0,0]:'5',
#          [0,0,0,0,0,0,0,0,1,0,0,0]:'6',
#          [0,0,0,0,0,0,0,0,0,1,0,0]:'7',
#          [0,0,0,0,0,0,0,0,0,0,1,0]:'8',
#          [0,0,0,0,0,0,0,0,0,0,0,1]:'9'}

In [0]:
def ceratDataSet(que,ans):
  #global = dict
  Q,A = [],[]
  for i in range(len(que)):
    a,b = [],[]
    for n in range(len(que[0])):
      a1 = dict1[que[i][n]]
      a = a+a1
    for k in range(len(ans[0])):
      a2 = dict1[ans[i][k]]
      b = b+a2
    Q.append(a)
    A.append(b)
  return np.array(Q),np.array(A)

In [0]:
class MyDataset(Dataset):
  def __init__(self, qua, ans):
    self.Q = qua
    self.A = ans

  def __len__(self):
    return len(self.A)

  def __getitem__(self, idx):
      q = self.Q[idx]
      a = self.A[idx]
      return q,a

In [0]:
que,ans = creatData(100000)

persentage = 0.8

Q,A = ceratDataSet(que,ans)
fron = int(len(Q)*persentage)


Q_train = Q[:fron]
A_train = A[:fron]

Q_test = Q[fron:]
A_test = A[fron:]

train_dataloader = DataLoader(MyDataset(Q_train,A_train),batch_size=100, shuffle=True)
test_dataloader = DataLoader(MyDataset(Q_test,A_test),batch_size=100, shuffle=True)

In [34]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.out1 = torch.nn.Linear(12*8,12*10)
    self.out2 = torch.nn.Linear(12*10,12*14)
    self.out3 = torch.nn.Linear(12*14,12*18)
    self.out4 = torch.nn.Linear(12*18,12*20)
    self.out5 = torch.nn.Linear(12*20,12*10)
    self.out6 = torch.nn.Linear(12*10,12*4)


  def forward(self, x):
    #print(self.out1.weight.type())
    #print(x.shape)
    x = self.out1(x)
    x = F.relu(x)
    x = self.out2(x)
    x = F.relu(x)
    x = self.out3(x)
    x = F.relu(x)
    x = self.out4(x)
    x = F.relu(x)
    x = self.out5(x)
    x = F.relu(x)
    x = self.out6(x)

    return x

In [49]:
model = CNN()

print(type(model))

if use_gpu:
  model = model.cuda()
print(model)

<class '__main__.CNN'>
CNN(
  (out1): Linear(in_features=96, out_features=120, bias=True)
  (out2): Linear(in_features=120, out_features=168, bias=True)
  (out3): Linear(in_features=168, out_features=216, bias=True)
  (out4): Linear(in_features=216, out_features=240, bias=True)
  (out5): Linear(in_features=240, out_features=120, bias=True)
  (out6): Linear(in_features=120, out_features=48, bias=True)
)


In [0]:
EPOCH = 100

loss_func = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [0]:
def train(epoch):
    for step, (x, y) in enumerate(train_dataloader):
        data = x.type(torch.cuda.FloatTensor)
        target = y.type(torch.cuda.FloatTensor)
        
        output = model(data)               # cnn output
#         print(output.type())
#         print(target.type())
        loss = loss_func(output, target)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, 
           step * len(data), len(train_dataloader.dataset),100. * step / len(train_dataloader), loss.data.item()))
    print("Finish")

In [52]:
for epoch in range(1,EPOCH):
    train(epoch)

Train Epoch: 1 [0/80000 (0%)]	Loss: 0.086843
Train Epoch: 1 [20000/80000 (25%)]	Loss: 0.053752
Train Epoch: 1 [40000/80000 (50%)]	Loss: 0.046170
Train Epoch: 1 [60000/80000 (75%)]	Loss: 0.045539
Finish
Train Epoch: 2 [0/80000 (0%)]	Loss: 0.041345
Train Epoch: 2 [20000/80000 (25%)]	Loss: 0.040074
Train Epoch: 2 [40000/80000 (50%)]	Loss: 0.039225
Train Epoch: 2 [60000/80000 (75%)]	Loss: 0.038840
Finish
Train Epoch: 3 [0/80000 (0%)]	Loss: 0.039780
Train Epoch: 3 [20000/80000 (25%)]	Loss: 0.040047
Train Epoch: 3 [40000/80000 (50%)]	Loss: 0.039093
Train Epoch: 3 [60000/80000 (75%)]	Loss: 0.037708
Finish
Train Epoch: 4 [0/80000 (0%)]	Loss: 0.036375
Train Epoch: 4 [20000/80000 (25%)]	Loss: 0.036099
Train Epoch: 4 [40000/80000 (50%)]	Loss: 0.034663
Train Epoch: 4 [60000/80000 (75%)]	Loss: 0.035064
Finish
Train Epoch: 5 [0/80000 (0%)]	Loss: 0.034496
Train Epoch: 5 [20000/80000 (25%)]	Loss: 0.031698
Train Epoch: 5 [40000/80000 (50%)]	Loss: 0.033577
Train Epoch: 5 [60000/80000 (75%)]	Loss: 0.0310

In [0]:
b = Variable(b).cuda().float
b = b.squeeze()
model(b)

In [0]:
def backtoNum(data):
  global dict2
  
  for i in range(len(data)):
    a = dict2[data[i]]
    p = p+a
  print(p)

In [0]:
backtoNum(test_dataloader.dataset[0])